# Online Hardware Validation

Interactive notebook for running **live** hardware validation tests.

**Test Categories:**
- Internal Noise (Shorted Inputs)
- External Noise (Floating Inputs)
- Known Signal Injection (KSI) - CH2-CH8
- Functional EEG Tests (Eyes Open/Closed/Blink)
- Eyes Open vs Closed Montage Comparison

**Architecture:**
- `acquisition.capture_live_stream()` - WebSocket frame streaming
- `analysis.run_pipeline()` - Same analysis as offline (with PDF export)

---
## Setup & Configuration

Run this cell first to import packages.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# IMPORTS
# ═══════════════════════════════════════════════════════════════════════════════
import asyncio
import json
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt

import nest_asyncio
nest_asyncio.apply()  # Allow nested asyncio in Jupyter

# Acquisition (live streaming)
from acquisition import HardwareClient, capture_live_stream

# Analysis (same as offline)
from analysis.pipeline import run_pipeline
from analysis.decode_bin import parse_framestream_bin
from analysis.plots import plot_eo_ec_publication_complete

print("✓ Imports successful")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# PROJECT ROOT FINDER
# ═══════════════════════════════════════════════════════════════════════════════

def find_project_root(marker="acquisition"):
    """Find project root by looking for marker directory."""
    p = Path().resolve()
    for parent in [p] + list(p.parents):
        if (parent / marker).exists():
            return parent
    raise FileNotFoundError(f"Could not find project root (marker: {marker})")

PROJECT_ROOT = find_project_root()
print(f"✓ Project root: {PROJECT_ROOT}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# HARDWARE PARAMETERS - EDIT IF NEEDED
# ═══════════════════════════════════════════════════════════════════════════════

SAMPLING_RATE_HZ = 16000    # ADS1299 sample rate (250, 500, 1000, 2000, 4000, 8000, 16000)
GAIN = 24                   # PGA gain (1, 2, 4, 6, 8, 12, 24)
MAINS_FREQ_HZ = 50.0        # Mains frequency for notch filter (50 Hz EU, 60 Hz US)

# Firmware/Board metadata (for sidecar JSON)
FIRMWARE = "FW2"
BOARD = "R2"

print(f"✓ Hardware parameters:")
print(f"  Sampling rate: {SAMPLING_RATE_HZ} Hz")
print(f"  Gain: {GAIN}")
print(f"  Mains frequency: {MAINS_FREQ_HZ} Hz")
print(f"  Firmware: {FIRMWARE}, Board: {BOARD}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# PROFILE MAPPING
# ═══════════════════════════════════════════════════════════════════════════════

PROFILES = {
    "INT": PROJECT_ROOT / "acquisition/profiles/all_shorted.json",
    "EXT": PROJECT_ROOT / "acquisition/profiles/all_normal.json",
    "KSI": PROJECT_ROOT / "acquisition/profiles/test_signal.json",
    "FUN": PROJECT_ROOT / "acquisition/profiles/eyes_open_closed.json",
}

print("✓ Profile mapping:")
for key, path in PROFILES.items():
    exists = "✓" if path.exists() else "✗"
    print(f"  {key}: {path.name} [{exists}]")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ESP32 & OUTPUT CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════════════

ESP32_IP = "node.local"           # ESP32 hostname or IP
CAPTURE_DURATION_S = 5.0          # Capture duration in seconds
SAVE_CAPTURES = True              # Save .bin files for later analysis
EXPORT_PDF = True                 # Generate PDF reports

# Output directory for saved files
OUTPUT_DIR = PROJECT_ROOT / "data/live"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Create hardware client
client = HardwareClient(ESP32_IP)

# Detect sample rate from hardware (override if detected)
detected_rate = client.detect_sample_rate()
if detected_rate:
    print(f"✓ Connected to {ESP32_IP}")
    print(f"  Detected sample rate: {detected_rate} Hz")
    if detected_rate != SAMPLING_RATE_HZ:
        print(f"  ⚠ Overriding configured rate ({SAMPLING_RATE_HZ}) with detected rate")
        SAMPLING_RATE_HZ = detected_rate
else:
    print(f"⚠ Could not detect sample rate, using configured: {SAMPLING_RATE_HZ} Hz")

print(f"  Output directory: {OUTPUT_DIR.absolute()}")
print(f"  Save captures: {SAVE_CAPTURES}")
print(f"  Export PDF: {EXPORT_PDF}")

---
## Helper: Capture & Analyze

Captures live data, saves to .bin file, and runs full analysis with PDF export.

In [ ]:
def capture_and_analyze(
    test_name: str,
    condition_code: str = "LIVE",
    duration_s: float = CAPTURE_DURATION_S,
    profile_path: Path = None,
):
    """
    Capture live data, save to file, and run full analysis pipeline.
    
    Args:
        test_name: Display name for the test (e.g., "Internal Noise")
        condition_code: Short code for filename (e.g., "INT", "EXT", "KSI")
        duration_s: Capture duration in seconds
        profile_path: Path to hardware profile JSON (optional). If provided,
                     applies the profile before capture.
    
    Returns:
        Tuple of (results_dict, bin_path) - bin_path is None if not saved
    """
    print(f"\n{'='*60}")
    print(f" {test_name} ".center(60, "="))
    print(f"{'='*60}\n")
    
    # Apply profile if provided
    if profile_path:
        print(f"[PROFILE] Applying: {profile_path.name}")
        success = client.load_and_apply_profile(str(profile_path))
        if success:
            client.dump_and_verify_registers()
        else:
            print(f"⚠ Profile application failed, continuing with current settings")
    
    # Capture live stream
    sample_bytes, raw_frames, info = asyncio.run(
        capture_live_stream(client, duration_s, SAMPLING_RATE_HZ)
    )
    
    if len(raw_frames) == 0:
        print("❌ No frames received - device not ready")
        return None, None
    
    print(f"✓ Captured {len(raw_frames):,} bytes")
    
    bin_path = None
    if SAVE_CAPTURES:
        # Validate frame alignment
        n_bytes = len(raw_frames)
        assert n_bytes % 1416 == 0, f"Unexpected bytes: {n_bytes} (not divisible by 1416)"
        n_frames = n_bytes // 1416
        
        # Build filename matching offline convention:
        # YYMMDD_HHMMSS_FW_BOARD_CONDITION_FRAMES_RATE.bin
        now = datetime.now()
        rate_k = SAMPLING_RATE_HZ // 1000
        
        filename = f"{now.strftime('%y%m%d_%H%M%S')}_{FIRMWARE}_{BOARD}_{condition_code}_{n_frames}_{rate_k}k.bin"
        bin_path = OUTPUT_DIR / filename
        
        # Save raw bytes
        bin_path.write_bytes(raw_frames)
        print(f"✓ Saved: {bin_path}")
        
        # Write sidecar JSON
        sidecar = {
            "timestamp": now.isoformat(),
            "esp32_ip": ESP32_IP,
            "firmware": FIRMWARE,
            "board": BOARD,
            "condition_code": condition_code,
            "profile_path": str(profile_path) if profile_path else None,
            "duration_s": duration_s,
            "fs_hz": SAMPLING_RATE_HZ,
            "gain": GAIN,
            "mains_freq_hz": MAINS_FREQ_HZ,
            "n_frames": n_frames,
            "n_bytes": n_bytes,
        }
        json_path = bin_path.with_suffix('.json')
        json_path.write_text(json.dumps(sidecar, indent=2))
        print(f"✓ Sidecar: {json_path.name}")
        
        # Run pipeline on saved file (gets PDF export!)
        results = run_pipeline(
            bin_path,
            display_plots=True,
            export_pdf_report=EXPORT_PDF,
            fs_hz=SAMPLING_RATE_HZ,
            gain=GAIN,
            mains_freq_hz=MAINS_FREQ_HZ,
        )
    else:
        # Run pipeline on bytes directly (no PDF)
        results = run_pipeline(
            raw_frames,
            source_name=test_name,
            display_plots=True,
            fs_hz=SAMPLING_RATE_HZ,
            gain=GAIN,
            mains_freq_hz=MAINS_FREQ_HZ,
        )
    
    print(f"\n✓ {test_name} complete.")
    return results, bin_path

---
## Internal Noise (Shorted Inputs)

Measures the intrinsic noise floor of the ADS1299 ADC with all inputs shorted.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# INTERNAL NOISE
# ═══════════════════════════════════════════════════════════════════════════════

results_internal, bin_internal = capture_and_analyze("Internal Noise", condition_code="INT", profile_path=PROFILES["INT"])

---
## External Noise (Floating Inputs)

Measures environmental noise pickup with floating (unconnected) inputs.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# EXTERNAL NOISE
# ═══════════════════════════════════════════════════════════════════════════════

results_external, bin_external = capture_and_analyze("External Noise", condition_code="EXT", profile_path=PROFILES["EXT"])

---
## Known Signal Injection (KSI)

Verifies channel integrity by injecting a known 40Hz signal into each channel.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# KSI CHANNEL 2
# ═══════════════════════════════════════════════════════════════════════════════

results_ksi_ch2, _ = capture_and_analyze("KSI 40Hz CH2", condition_code="KSI_CH2_40HZ", profile_path=PROFILES["KSI"])

In [ ]:
# KSI Channel 3
results_ksi_ch3, _ = capture_and_analyze("KSI 40Hz CH3", condition_code="KSI_CH3_40HZ", profile_path=PROFILES["KSI"])

In [ ]:
# KSI Channel 4
results_ksi_ch4, _ = capture_and_analyze("KSI 40Hz CH4", condition_code="KSI_CH4_40HZ", profile_path=PROFILES["KSI"])

In [ ]:
# KSI Channel 5
results_ksi_ch5, _ = capture_and_analyze("KSI 40Hz CH5", condition_code="KSI_CH5_40HZ", profile_path=PROFILES["KSI"])

In [ ]:
# KSI Channel 6
results_ksi_ch6, _ = capture_and_analyze("KSI 40Hz CH6", condition_code="KSI_CH6_40HZ", profile_path=PROFILES["KSI"])

In [ ]:
# KSI Channel 7
results_ksi_ch7, _ = capture_and_analyze("KSI 40Hz CH7", condition_code="KSI_CH7_40HZ", profile_path=PROFILES["KSI"])

In [ ]:
# KSI Channel 8
results_ksi_ch8, _ = capture_and_analyze("KSI 40Hz CH8", condition_code="KSI_CH8_40HZ", profile_path=PROFILES["KSI"])

---
## Functional EEG Tests

Real EEG recordings from a human subject.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# EYES OPEN
# ═══════════════════════════════════════════════════════════════════════════════

input("Subject: Keep eyes OPEN. Press Enter to start recording...")
results_eo, bin_eo = capture_and_analyze("Eyes Open", condition_code="FUNEO", profile_path=PROFILES["FUN"])

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# EYES CLOSED
# ═══════════════════════════════════════════════════════════════════════════════

input("Subject: Keep eyes CLOSED. Press Enter to start recording...")
results_ec, bin_ec = capture_and_analyze("Eyes Closed", condition_code="FUNEC", profile_path=PROFILES["FUN"])

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# EYE BLINKS
# ═══════════════════════════════════════════════════════════════════════════════

input("Subject: Blink every 2-3 seconds. Press Enter to start recording...")
results_blink, _ = capture_and_analyze("Eye Blinks", condition_code="FUNEB", profile_path=PROFILES["FUN"])

---
## Eyes Open vs Closed Montage Comparison

Side-by-side comparison showing alpha modulation.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# EO vs EC - COMPLETE PUBLICATION FIGURE (3 panels)
# ═══════════════════════════════════════════════════════════════════════════════

# Check if EO/EC captures were saved
if bin_eo and bin_ec and bin_eo.exists() and bin_ec.exists():
    print("="*80)
    print(" EYES OPEN vs EYES CLOSED — PUBLICATION FIGURE ".center(80, "="))
    print("="*80)

    # Parse both files
    print("\n[1/3] Parsing Eyes Open...")
    bin_bytes_eo = bin_eo.read_bytes()
    counts_eo, meta_eo = parse_framestream_bin(bin_bytes_eo)
    print(f"      {meta_eo['n_frames']} frames")

    print("[2/3] Parsing Eyes Closed...")
    bin_bytes_ec = bin_ec.read_bytes()
    counts_ec, meta_ec = parse_framestream_bin(bin_bytes_ec)
    print(f"      {meta_ec['n_frames']} frames")

    # ─── COMPLETE 3-PANEL FIGURE ──────────────────────────────────────────────────
    print("[3/3] Generating complete publication figure...")
    print("      (a) Electrode montage")
    print("      (b) PSD comparison (linear axes)")
    print("      (c) EEG time series (EO|EC)")

    fig = plot_eo_ec_publication_complete(
        counts_eo, counts_ec,
        fs_hz=SAMPLING_RATE_HZ,
        duration_s=5.0,
        psd_channel=5,  # O2 - best for alpha
        montage_image_path=str(PROJECT_ROOT / "analysis/assets/montage_head.png"),
    )
    plt.show()
else:
    print("⚠ Eyes Open and/or Eyes Closed captures not available.")
    print("  Run the Eyes Open and Eyes Closed cells above first.")

---
## Custom Test

Run any custom test by specifying a name and condition code.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CUSTOM TEST
# ═══════════════════════════════════════════════════════════════════════════════

TEST_NAME = "Custom Test"   # ← EDIT THIS
CONDITION = "CUSTOM"        # ← EDIT THIS (for filename)
DURATION = 5.0              # ← EDIT THIS (seconds)
PROFILE = None              # ← EDIT: PROFILES["INT"], PROFILES["KSI"], etc.

results_custom, _ = capture_and_analyze(TEST_NAME, condition_code=CONDITION, duration_s=DURATION, profile_path=PROFILE)